In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

COVID = pd.read_excel('./owid-covid-data.xlsx')
COVID = COVID[COVID['location']=='South Korea']
COVID = COVID.fillna(0)
COVID.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
154398,KOR,Asia,South Korea,2020-01-22,1.0,0.0,0.0,0.0,0.0,0.0,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154399,KOR,Asia,South Korea,2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154400,KOR,Asia,South Korea,2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154401,KOR,Asia,South Korea,2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154402,KOR,Asia,South Korea,2020-01-26,3.0,1.0,0.0,0.0,0.0,0.0,...,6.2,40.9,0.0,12.27,83.03,0.916,43.4,0.17,2.47,0.845918


In [4]:
COVID.tail()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
155231,KOR,Asia,South Korea,2022-05-04,17438068.0,42277.0,50206.000,23158.0,79.0,98.857,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.0,0.0,0.0
155232,KOR,Asia,South Korea,2022-05-05,17464782.0,26714.0,38595.143,23206.0,48.0,68.857,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.0,0.0,0.0
155233,KOR,Asia,South Korea,2022-05-06,17504334.0,39552.0,38065.143,23289.0,83.0,70.714,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.0,0.0,0.0
155234,KOR,Asia,South Korea,2022-05-07,17544398.0,40064.0,38392.714,23360.0,71.0,69.286,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.0,0.0,0.0
155235,KOR,Asia,South Korea,2022-05-08,17564999.0,20601.0,38466.571,23400.0,40.0,63.143,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.0,0.0,0.0


In [13]:
COVID = COVID[['total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed']]

In [14]:
COVID.head()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed
154398,1.0,0.0,0.0,0.0,0.0,0.0
154399,1.0,0.0,0.0,0.0,0.0,0.0
154400,2.0,1.0,0.0,0.0,0.0,0.0
154401,2.0,0.0,0.0,0.0,0.0,0.0
154402,3.0,1.0,0.0,0.0,0.0,0.0


In [16]:
COVID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838 entries, 154398 to 155235
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   total_cases          838 non-null    float64
 1   new_cases            838 non-null    float64
 2   new_cases_smoothed   838 non-null    float64
 3   total_deaths         838 non-null    float64
 4   new_deaths           838 non-null    float64
 5   new_deaths_smoothed  838 non-null    float64
dtypes: float64(6)
memory usage: 78.1 KB


In [17]:
X = COVID.drop(['new_deaths'], axis=1)
y = COVID['new_deaths']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

estimators = [
    ('scalor', StandardScaler()),
    ('clf', DecisionTreeClassifier())
]

pipe = Pipeline(estimators)

In [19]:
pipe.set_params(clf__max_depth=2)
pipe.set_params(clf__random_state=13)

Pipeline(steps=[('scalor', StandardScaler()),
                ('clf', DecisionTreeClassifier(max_depth=2, random_state=13))])

In [21]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scalor', StandardScaler()),
                ('clf', DecisionTreeClassifier(max_depth=2, random_state=13))])

In [22]:
from sklearn.metrics import accuracy_score

y_pred_tr = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)

print('Train acc :', accuracy_score(y_train, y_pred_tr))
print('Test acc :', accuracy_score(y_test, y_pred_test))

Train acc : 0.2373134328358209
Test acc : 0.19642857142857142


In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_depth': [i for i in range(4, 14, 2)]}
be_tree = DecisionTreeClassifier(max_depth=2, random_state=13)

In [25]:
GridSearch = GridSearchCV(estimator=be_tree, param_grid=params, cv=5)
GridSearch.fit(X, y)

C:\Users\nihso\miniconda3\envs\ds_study\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=2, random_state=13),
             param_grid={'max_depth': [4, 6, 8, 10, 12]})

In [26]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(GridSearch.cv_results_)

{   'mean_fit_time': array([0.0056047 , 0.00600543, 0.00600438, 0.00640383, 0.0072062 ]),
    'mean_score_time': array([0.00140162, 0.00240293, 0.0014019 , 0.00140386, 0.00120149]),
    'mean_test_score': array([0.14799686, 0.11576846, 0.10379954, 0.09184488, 0.07988309]),
    'param_max_depth': masked_array(data=[4, 6, 8, 10, 12],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object),
    'params': [   {'max_depth': 4},
                  {'max_depth': 6},
                  {'max_depth': 8},
                  {'max_depth': 10},
                  {'max_depth': 12}],
    'rank_test_score': array([1, 2, 3, 4, 5]),
    'split0_test_score': array([0.19642857, 0.16666667, 0.17857143, 0.16666667, 0.17857143]),
    'split1_test_score': array([0.10119048, 0.11904762, 0.10714286, 0.08928571, 0.08928571]),
    'split2_test_score': array([0.125     , 0.04761905, 0.04166667, 0.05357143, 0.0297619 ]),
    'split3_test_score': array([0.1497006 , 0.149

In [36]:
COVIDdr = COVID.drop(['new_deaths', 'new_deaths_smoothed'], axis=1)

In [37]:
import statsmodels.formula.api as smf

lm_model = smf.ols(formula='y ~ X', data=COVIDdr).fit()

In [38]:
COVID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838 entries, 154398 to 155235
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   total_cases          838 non-null    float64
 1   new_cases            838 non-null    float64
 2   new_cases_smoothed   838 non-null    float64
 3   total_deaths         838 non-null    float64
 4   new_deaths           838 non-null    float64
 5   new_deaths_smoothed  838 non-null    float64
dtypes: float64(6)
memory usage: 78.1 KB


In [39]:
X_train

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths_smoothed
154604,15318.0,279.0,102.857,305.0,0.000
155075,447230.0,3030.0,3754.286,3624.0,42.286
155217,16674045.0,90825.0,99283.429,21667.0,187.857
155135,811122.0,17540.0,12121.143,6712.0,26.143
155080,473034.0,5127.0,4591.143,3852.0,43.429
...,...,...,...,...,...
155140,907214.0,22904.0,18531.000,6812.0,22.571
154926,160084.0,742.0,716.143,2026.0,1.857
154472,10237.0,81.0,93.429,183.0,4.429
154574,13672.0,60.0,47.714,293.0,0.714


In [40]:
lm_model.params

Intercept    0.451233
X[0]        -0.000001
X[1]         0.000366
X[2]        -0.000236
X[3]         0.000052
X[4]         0.943708
dtype: float64

In [44]:
COVID

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
154398,KOR,Asia,South Korea,2020-01-22,1.0,0.0,0.000,0.0,0.0,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154399,KOR,Asia,South Korea,2020-01-23,1.0,0.0,0.000,0.0,0.0,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154400,KOR,Asia,South Korea,2020-01-24,2.0,1.0,0.000,0.0,0.0,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154401,KOR,Asia,South Korea,2020-01-25,2.0,0.0,0.000,0.0,0.0,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154402,KOR,Asia,South Korea,2020-01-26,3.0,1.0,0.000,0.0,0.0,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,43.4,0.17,2.47,0.845918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155231,KOR,Asia,South Korea,2022-05-04,17438068.0,42277.0,50206.000,23158.0,79.0,98.857,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155232,KOR,Asia,South Korea,2022-05-05,17464782.0,26714.0,38595.143,23206.0,48.0,68.857,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155233,KOR,Asia,South Korea,2022-05-06,17504334.0,39552.0,38065.143,23289.0,83.0,70.714,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155234,KOR,Asia,South Korea,2022-05-07,17544398.0,40064.0,38392.714,23360.0,71.0,69.286,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000


In [45]:
COVID.drop(['iso_code','continent','location','date'], axis=1,inplace=True)
COVID.drop(['tests_units'], axis=1, inplace=True)

In [47]:
COVID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838 entries, 154398 to 155235
Data columns (total 62 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   total_cases                                 838 non-null    float64
 1   new_cases                                   838 non-null    float64
 2   new_cases_smoothed                          838 non-null    float64
 3   total_deaths                                838 non-null    float64
 4   new_deaths                                  838 non-null    float64
 5   new_deaths_smoothed                         838 non-null    float64
 6   total_cases_per_million                     838 non-null    float64
 7   new_cases_per_million                       838 non-null    float64
 8   new_cases_smoothed_per_million              838 non-null    float64
 9   total_deaths_per_million                    838 non-null    float64
 10  new_de

In [138]:
X = COVID.drop(['new_deaths','total_deaths','new_deaths_per_million', 'new_deaths_smoothed', 'new_deaths_smoothed_per_million'], axis=1)
y = COVID['new_deaths']

In [112]:
lm = sm.OLS(y, X).fit()

In [113]:
lm.rsquared

0.9361316361045138

In [114]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             new_deaths   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     299.9
Date:                Tue, 10 May 2022   Prob (F-statistic):               0.00
Time:                        13:39:59   Log-Likelihood:                -3578.5
No. Observations:                 838   AIC:                             7237.
Df Residuals:                     798   BIC:                             7426.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
total_cases                                   18.8076     41.608      0.452      0.651     -62.867     100.482
new_cases                                    -40.5769     42.249     -0.960      0.337    -123.509      42.355
new_cases_smoothed                           -11.4190     43.007     -0.266      0.791     -95.840      73.002
total_cases_per_million                     -964.9283   2134.710     -0.452      0.651   -5155.238    3225.381
new_cases_per_million                       2081.8189   2167.593      0.960      0.337   -2173.040    6336.677
new_cases_smoothed_per_million               585.8752   2206.502      0.266      0.791   -3745.359    4917.110
total_deaths_per_million                      -0.5403      0.115     -4.711      0.000      -0.765      -0.315
reproduction_rate                             -0.1052      1.381     -0.076      0.939      -2.816       2.605
icu_patients                                  28.5075     44.760      0.637      0.524     -59.355     116.370
icu_patients_per_million                   -1459.2043   2296.428     -0.635      0.525   -5966.958    3048.550
hosp_patients                                  0.1444      0.176      0.823      0.411      -0.200       0.489
hosp_patients_per_million                      0.2511      0.325      0.771      0.441      -0.388       0.890
weekly_icu_admissions                        196.2604    244.867      0.801      0.423    -284.400     676.920
weekly_icu_admissions_per_million          -1.007e+04   1.26e+04     -0.802      0.423   -3.47e+04    1.46e+04
weekly_hosp_admissions                      -273.6617    100.504     -2.723      0.007    -470.945     -76.378
weekly_hosp_admissions_per_million          1.404e+04   5156.378      2.723      0.007    3918.784    2.42e+04
total_tests                                    0.0139      0.062      0.223      0.824      -0.109       0.137
new_tests                                     -0.0070      0.061     -0.115      0.909      -0.127       0.113
total_tests_per_thousand                    -714.0976   3204.433     -0.223      0.824   -7004.211    5576.016
new_tests_per_thousand                       359.7283   3142.643      0.114      0.909   -5809.096    6528.552
new_tests_smoothed                            -0.0702      0.060     -1.176      0.240      -0.187       0.047
new_tests_smoothed_per_thousand             3594.0423   3060.578      1.174      0.241   -2413.693    9601.777
positive_rate                                  9.6596      3.657      2.642      0.008       2.482      16.838
tests_per_case                                 0.0010      0.004      0.226      0.821      -0.007       0.009
total_vaccinations                             0.0010      0.001      1.700      0.089

In [115]:
death = lm.predict(X)

In [118]:
COVIDNew = COVID[['total_cases', 'new_cases', 'new_cases_smoothed','reproduction_rate' , 'hosp_patients','positive_rate' , 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'stringency_index','human_development_index' , 'excess_mortality_cumulative_per_million', 'new_people_vaccinated_smoothed']]

In [119]:
X = COVIDNew
y = COVID['new_deaths']

In [120]:
lm = sm.OLS(y, X).fit()

In [121]:
lm.rsquared

0.9021122641664835

In [122]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             new_deaths   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     633.6
Date:                Tue, 10 May 2022   Prob (F-statistic):               0.00
Time:                        13:59:45   Log-Likelihood:                -3757.4
No. Observations:                 838   AIC:                             7541.
Df Residuals:                     825   BIC:                             7602.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
total_cases                              4.936e-06   4.01e-07     12.295      0.000    4.15e-06    5.72e-06
new_cases                                   0.0001   4.44e-05      2.670      0.008    3.14e-05       0.000
new_cases_smoothed                          0.0006    4.9e-05     12.791      0.000       0.001       0.001
reproduction_rate                          -4.1170      1.596     -2.580      0.010      -7.249      -0.985
hosp_patients                           -2.027e-11   2.07e-11     -0.981      0.327   -6.08e-11    2.03e-11
positive_rate                               4.1631      2.751      1.513      0.131      -1.236       9.562
total_vaccinations                      -2.025e-05   7.79e-06     -2.598      0.010   -3.55e-05   -4.95e-06
people_vaccinated                        1.745e-05   7.19e-06      2.428      0.015    3.34e-06    3.16e-05
people_fully_vaccinated                   2.34e-05   8.17e-06      2.865      0.004    7.37e-06    3.94e-05
total_boosters                           1.908e-05   7.77e-06      2.454      0.014    3.82e-06    3.43e-05
stringency_index                            0.3901      0.075      5.180      0.000       0.242       0.538
human_development_index                   -15.4029      4.615     -3.338      0.001     -24.461      -6.345
excess_mortality_cumulative_per_million    -0.0510      0.032     -1.595      0.111      -0.114       0.012
new_people_vaccinated_smoothed           2.128e-05   1.23e-05      1.728      0.084   -2.89e-06    4.54e-05
==============================================================================
Omnibus:                      228.470   Durbin-Watson:                   1.470
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6938.819
Skew:                           0.562   Prob(JB):                         0.00
Kurtosis:                      17.052   Cond. No.                     1.15e+20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.66e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [123]:
dead= lm.predict(X)

In [124]:
COVIDNew['Predict Death'] = dead

C:\Users\nihso\AppData\Local\Temp\ipykernel_3800\3170820403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  COVIDNew['Predict Death'] = dead


In [125]:
COVIDNew['Predict with all'] = death
COVIDNew['Real death'] = COVID['new_deaths']

C:\Users\nihso\AppData\Local\Temp\ipykernel_3800\3587457536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  COVIDNew['Predict with all'] = death
C:\Users\nihso\AppData\Local\Temp\ipykernel_3800\3587457536.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  COVIDNew['Real death'] = COVID['new_deaths']


In [126]:
COVIDNew

,total_cases,new_cases,new_cases_smoothed,reproduction_rate,hosp_patients,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,stringency_index,human_development_index,excess_mortality_cumulative_per_million,new_people_vaccinated_smoothed,Predict Death,Predict with all,Real death
154398,1.0,0.0,0.000,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.916,0.000000,0.0,-14.109027,-2.737713,0.0
154399,1.0,0.0,0.000,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.916,0.000000,0.0,-14.109027,-2.737713,0.0
154400,2.0,1.0,0.000,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.916,0.000000,0.0,-14.108903,-4.250972,0.0
154401,2.0,0.0,0.000,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.916,0.000000,0.0,-14.109022,-3.228646,0.0
154402,3.0,1.0,0.000,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.916,0.845918,0.0,-14.152019,-3.858791,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155231,17438068.0,42277.0,50206.000,0.57,0.0,0.3627,123823522.0,45035707.0,44552150.0,35751825.0,0.0,0.916,0.000000,1110.0,110.672753,56.253004,79.0
155232,17464782.0,26714.0,38595.143,0.44,0.0,0.0000,123827158.0,45035755.0,44552198.0,35755365.0,0.0,0.916,0.000000,962.0,100.698427,72.923438,48.0
155233,17504334.0,39552.0,38065.143,0.00,0.0,0.0000,124081608.0,45037744.0,44554768.0,36005262.0,0.0,0.916,0.000000,832.0,103.600706,78.967404,83.0
155234,17544398.0,40064.0,38392.714,0.00,0.0,0.0000,124131394.0,45038345.0,44555511.0,36053704.0,0.0,0.916,0.000000,737.0,104.006317,68.815215,71.0


In [135]:
lm.aic

7540.833260222283

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [130]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [131]:
from sklearn.metrics import mean_squared_error

pred_tr = reg.predict(X_train)
pred_test = reg.predict(X_test)

rmse_tr = np.sqrt(mean_squared_error(y_train, pred_tr))
rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))

In [134]:
print('RMSE of Train Data: ', rmse_tr)
print('RMSE of Test Data: ', rmse_test)

RMSE of Train Data:  17.128038438119574
RMSE of Test Data:  18.503502080077805


In [140]:
def stepwise_feature_selection(X_train, y_train, variables=X_train.columns.tolist() ): 
    import statsmodels.api as sm 
    import matplotlib.pyplot as plt 
    import warnings 
    warnings.filterwarnings("ignore") 
    y = y_train ## 반응 변수 
    selected_variables = [] ## 선택된 변수들 
    sl_enter = 0.05 
    sl_remove = 0.05 
    sv_per_step = [] ## 각 스텝별로 선택된 변수들 
    adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수 
    steps = [] ## 스텝 
    step = 0 
    while len(variables) > 0: 
        remainder = list(set(variables) - set(selected_variables)) 
        pval = pd.Series(index=remainder) ## 변수의 p-value ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 ## 선형 모형을 적합한다. 
        for col in remainder: 
            X = X_train[selected_variables+[col]] 
            X = sm.add_constant(X) 
            model = sm.OLS(y,X).fit(disp=0) 
            pval[col] = model.pvalues[col] 
            min_pval = pval.min() 
        if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함 
            selected_variables.append(pval.idxmin()) ## 선택된 변수들에대해서 ## 어떤 변수를 제거할지 고른다. 
            while len(selected_variables) > 0: 
                selected_X = X_train[selected_variables] 
                selected_X = sm.add_constant(selected_X) 
                selected_pval = sm.OLS(y,selected_X).fit(disp=0).pvalues[1:] ## 절편항의 p-value는 뺀다 
                max_pval = selected_pval.max() 
                if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외 
                    remove_variable = selected_pval.idxmax() 
                    selected_variables.remove(remove_variable) 
                else: 
                    break 
            step += 1 
            steps.append(step) 
            adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit(disp=0).rsquared_adj 
            adjusted_r_squared.append(adj_r_squared) 
            sv_per_step.append(selected_variables.copy()) 
        else: 
            break 
    fig = plt.figure(figsize=(100,10)) 
    fig.set_facecolor('white') 
    font_size = 15 
    plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12) 
    
    plt.plot(steps,adjusted_r_squared, marker='o') 
    
    plt.ylabel('Adjusted R Squared',fontsize=font_size) 
    plt.grid(True) 
    plt.show() 

    return selected_variables 
selected_variables = stepwise_feature_selection(X_train, y_train)



In [137]:
model = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train[selected_variables]))).fit(disp=0) 
print(model.summary())

NameError: name 'selected_variables' is not defined

In [128]:
COVID

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
154398,1.0,0.0,0.000,0.0,0.0,0.000,0.019,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154399,1.0,0.0,0.000,0.0,0.0,0.000,0.019,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154400,2.0,1.0,0.000,0.0,0.0,0.000,0.039,0.019,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154401,2.0,0.0,0.000,0.0,0.0,0.000,0.039,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154402,3.0,1.0,0.000,0.0,0.0,0.000,0.058,0.019,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,43.4,0.17,2.47,0.845918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155231,17438068.0,42277.0,50206.000,23158.0,79.0,98.857,339889.006,824.030,978.576,451.377,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155232,17464782.0,26714.0,38595.143,23206.0,48.0,68.857,340409.694,520.688,752.266,452.313,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155233,17504334.0,39552.0,38065.143,23289.0,83.0,70.714,341180.611,770.916,741.936,453.931,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155234,17544398.0,40064.0,38392.714,23360.0,71.0,69.286,341961.506,780.896,748.320,455.315,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000


In [91]:
COVID

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
154398,1.0,0.0,0.000,0.0,0.0,0.000,0.019,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154399,1.0,0.0,0.000,0.0,0.0,0.000,0.019,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154400,2.0,1.0,0.000,0.0,0.0,0.000,0.039,0.019,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154401,2.0,0.0,0.000,0.0,0.0,0.000,0.039,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
154402,3.0,1.0,0.000,0.0,0.0,0.000,0.058,0.019,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,43.4,0.17,2.47,0.845918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155231,17438068.0,42277.0,50206.000,23158.0,79.0,98.857,339889.006,824.030,978.576,451.377,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155232,17464782.0,26714.0,38595.143,23206.0,48.0,68.857,340409.694,520.688,752.266,452.313,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155233,17504334.0,39552.0,38065.143,23289.0,83.0,70.714,341180.611,770.916,741.936,453.931,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
155234,17544398.0,40064.0,38392.714,23360.0,71.0,69.286,341961.506,780.896,748.320,455.315,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000


In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

corona_tree = DecisionTreeClassifier(max_depth=4, random_state=13)
corona_tree.fit(X_train, y_train)

y_pred_tr = corona_tree.predict(X_train)
y_pred_test = corona_tree.predict(X_test)

print('Train Acc: ', accuracy_score(y_train, y_pred_tr))
print('Test Acc: ',accuracy_score(y_test, y_pred_test))

Train Acc:  0.2791044776119403
Test Acc:  0.14285714285714285


In [72]:
dict(zip(X_train.columns, corona_tree.feature_importances_))

{'total_cases': 0.24454901747679164,
 'new_cases': 0.0,
 'new_cases_smoothed': 0.5201863523830745,
 'reproduction_rate': 0.07901137359344072,
 'hosp_patients': 0.0,
 'total_vaccinations': 0.07456561907905652,
 'people_vaccinated': 0.0,
 'people_fully_vaccinated': 0.0,
 'total_boosters': 0.0,
 'stringency_index': 0.06235627927623016,
 'excess_mortality_cumulative_absolute': 0.019331358191406176}

In [100]:
X = COVID.drop(['new_deaths','total_deaths','new_deaths_per_million', 'new_deaths_smoothed', 'new_deaths_smoothed_per_million', 'icu_patients_per_million'], axis=1)
y = COVID['new_deaths']

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

corona_tree = DecisionTreeClassifier(max_depth=4, random_state=13)
corona_tree.fit(X_train, y_train)

y_pred_tr = corona_tree.predict(X_train)
y_pred_test = corona_tree.predict(X_test)

print('Train Acc: ', accuracy_score(y_train, y_pred_tr))
print('Test Acc: ',accuracy_score(y_test, y_pred_test))

Train Acc:  0.2626865671641791
Test Acc:  0.17261904761904762


In [102]:
dict(zip(X_train.columns, corona_tree.feature_importances_))

{'total_cases': 0.0,
 'new_cases': 0.031120004730377085,
 'new_cases_smoothed': 0.03572469872488641,
 'total_cases_per_million': 0.0,
 'new_cases_per_million': 0.0,
 'new_cases_smoothed_per_million': 0.1085658306011381,
 'total_deaths_per_million': 0.0,
 'reproduction_rate': 0.0,
 'icu_patients': 0.5731187368688394,
 'hosp_patients': 0.0,
 'hosp_patients_per_million': 0.0,
 'weekly_icu_admissions': 0.0,
 'weekly_icu_admissions_per_million': 0.0,
 'weekly_hosp_admissions': 0.0,
 'weekly_hosp_admissions_per_million': 0.0,
 'total_tests': 0.07504998789265442,
 'new_tests': 0.0,
 'total_tests_per_thousand': 0.0,
 'new_tests_per_thousand': 0.0,
 'new_tests_smoothed': 0.025571211498440875,
 'new_tests_smoothed_per_thousand': 0.0,
 'positive_rate': 0.0,
 'tests_per_case': 0.0,
 'total_vaccinations': 0.0,
 'people_vaccinated': 0.0,
 'people_fully_vaccinated': 0.026636678644209273,
 'total_boosters': 0.04492519202877773,
 'new_vaccinations': 0.0,
 'new_vaccinations_smoothed': 0.0,
 'total_vacci